<a href="https://colab.research.google.com/github/joissriram99/Sample/blob/master/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install d2l

     |████████████████████████████████| 61kB 2.5MB/s 


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn

In [ ]:
class Reshape(nn.Module):
  def forward(self,x):
    return x.reshape(-1,1,28,28)

net=torch.nn.Sequential(Reshape(),
                  nn.Conv2d(1,6,kernel_size=5,padding=2),nn.ReLU(),
                  nn.AvgPool2d(kernel_size=2,stride=2),
                  nn.Conv2d(6,16,kernel_size=5),nn.ReLU(),
                  nn.AvgPool2d(kernel_size=2,stride=2),
                  nn.Flatten(),
                  nn.Linear(16*5*5,120),nn.ReLU(),
                  nn.Linear(120,84),nn.ReLU(),
                  nn.Linear(84,10))

In [ ]:
x=torch.rand(1,1,28,28,dtype=torch.float32)

for layer in net:
  x=layer(x)
  print(layer.__class__.__name__,'output shape \t',x.shape)

Reshape output shape 	 torch.Size([1, 1, 28, 28])
Conv2d output shape 	 torch.Size([1, 6, 28, 28])
ReLU output shape 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape 	 torch.Size([1, 6, 14, 14])
Conv2d output shape 	 torch.Size([1, 16, 10, 10])
ReLU output shape 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape 	 torch.Size([1, 16, 5, 5])
Flatten output shape 	 torch.Size([1, 400])
Linear output shape 	 torch.Size([1, 120])
ReLU output shape 	 torch.Size([1, 120])
Linear output shape 	 torch.Size([1, 84])
ReLU output shape 	 torch.Size([1, 84])
Linear output shape 	 torch.Size([1, 10])


In [ ]:
def acc_on_gpu(net, data_iter, device=None):
  net.eval()
  if not device:
    device = next(iter(net.parameters())).device
  metric=d2l.Accumulator(2)
  for X,y in data_iter:
    X,y=X.to(device),y.to(device)
    metric.add(d2l.accuracy(net(X),y),d2l.size(y))
  return metric[0]/metric[1]



In [ ]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw


Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def train123(net,train_iter,test_iter,num_epochs,lr,device=d2l.try_gpu()):
  def init_weig(m):
    if type(m)==nn.Linear or type(m)==nn.Conv2d:
      torch.nn.init.xavier_uniform_(m.weight)
  net.apply(init_weig)
  net.to(device)
  for epoch in range(num_epochs):
    metric = d2l.Accumulator(3)
    net.train()
    optimizer=torch.optim.SGD(net.parameters(),lr=lr)
    loss=nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[0, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer=d2l.Timer()
    for i,(X,y) in enumerate(train_iter):
      timer.start()
      optimizer.zero_grad()
      X,y=X.to(device),y.to(device)
      y_hat=net(X)
      l=loss(y_hat,y)
      l.backward()
      optimizer.step()
      with torch.no_grad():
        metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
      timer.stop()
      train_loss = metric[0]/metric[2]
      train_acc = metric[1]/metric[2]
      if (i + 1) % 50 == 0:
          animator.add(epoch + i / len(train_iter),
                        (train_loss, train_acc, None))
      test_acc = acc_on_gpu(net, test_iter)
      animator.add(epoch+1, (None, None, test_acc))
  print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, '
              f'test acc {test_acc:.3f}')
  print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')




In [ ]:
lr, num_epochs = 0.9, 10
train123(net, train_iter, test_iter, num_epochs, lr)
